In [1]:
# ----------------------------------------------------------
# Book Recommendation Engine using K-Nearest Neighbors (KNN)
# ----------------------------------------------------------

# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [2]:
# get data files
books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

In [3]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [4]:
# add your code here - consider creating a new cell for each section of code

df_ratings.info()

df_ratings.head()

df_ratings.user.unique()

df_books.info()

df_books.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1149780 entries, 0 to 1149779
Data columns (total 3 columns):
 #   Column  Non-Null Count    Dtype  
---  ------  --------------    -----  
 0   user    1149780 non-null  int32  
 1   isbn    1149780 non-null  object 
 2   rating  1149780 non-null  float32
dtypes: float32(1), int32(1), object(1)
memory usage: 17.5+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271379 entries, 0 to 271378
Data columns (total 3 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   isbn    271379 non-null  object
 1   title   271379 non-null  object
 2   author  271377 non-null  object
dtypes: object(3)
memory usage: 6.2+ MB


,isbn,title,author
0,0195153448,Classical Mythology,Mark P. O. Morford
1,0002005018,Clara Callan,Richard Bruce Wright
2,0060973129,Decision in Normandy,Carlo D'Este
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata
4,0393045218,The Mummies of Urumchi,E. J. W. Barber


In [5]:
df = df_ratings

count1 = df['user'].value_counts()
count2 = df['isbn'].value_counts()

# Get users with at least 200 entries
valid_users = count1[count1 >= 200].index

# Get ISBNs with at least 100 entries
valid_isbns = count2[count2 >= 100].index

# Filter the DataFrame
df = df[df['user'].isin(valid_users) & df['isbn'].isin(valid_isbns)]


df = pd.merge(right=df, left = df_books, on="isbn")

df = df.drop_duplicates(["title", "user"])

piv = df.pivot(index='title', columns='user', values='rating').fillna(0)

matrix = piv.values

matrix.shape

piv

from sklearn.neighbors import NearestNeighbors
model_knn=NearestNeighbors(metric='cosine',algorithm='brute')
model_knn.fit(matrix)

NearestNeighbors(algorithm='brute', metric='cosine')

In [6]:
# function to return recommended books - this will be tested
def get_recommends(book=""):
    # Get the data for the given book
    book_data = piv.loc[book].values.reshape(1, -1)

    # Find the 6 closest books (including the book itself)
    distances, indices = model_knn.kneighbors(book_data, n_neighbors=6)

    # Create a list to store recommended books
    recommended = []

    # Loop through the results using index
    for i in range(len(distances[0])):
        dist = distances[0][i]
        idx = indices[0][i]

        if dist != 0:  # Skip the book itself
            book_name = piv.index[idx]
            recommended.append([book_name, dist])

    # Return the book and the recommendations (reversed for ranking)
    return [book, recommended[::-1]]


get_recommends('The Queen of the Damned (Vampire Chronicles (Paperback))')

['The Queen of the Damned (Vampire Chronicles (Paperback))',
 [['Catch 22', np.float32(0.7939835)],
  ['The Witching Hour (Lives of the Mayfair Witches)', np.float32(0.74486566)],
  ['Interview with the Vampire', np.float32(0.73450685)],
  ['The Tale of the Body Thief (Vampire Chronicles (Paperback))',
   np.float32(0.53763384)],
  ['The Vampire Lestat (Vampire Chronicles, Book II)',
   np.float32(0.51784116)]]]

In [7]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2):
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", [["I'll Be Seeing You", np.float32(0.8016211)], ['The Weight of Water', np.float32(0.77085835)], ['The Surgeon', np.float32(0.7699411)], ['I Know This Much Is True', np.float32(0.7677075)], ['The Lovely Bones: A Novel', np.float32(0.7234864)]]]
You passed the challenge! 🎉🎉🎉🎉🎉
